In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
ssc.checkpoint("C:/Users/super/Documents/Documentos Escolares/ITAM/Séptimo Semestre/Bases de Datos No Relacionales/BDnR_GIT/ProyectoFinal/PruebaSpark")

In [5]:
socket_stream = ssc.socketTextStream("192.168.254.1", 5555)

In [6]:
lines = socket_stream.window(20)

In [7]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [8]:
( lines.flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("#") )
  .map( lambda word: ( word.lower(), 1 ) )
  .reduceByKey( lambda a, b: a + b )
  .map( lambda rec: Tweet( rec[0], rec[1] ) )
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
              .limit(10).registerTempTable("tweets") ) )

In [9]:
sqlContext

In [10]:
ssc.start()

In [11]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

In [13]:
import time
from IPython import display


count = 0
while count < 10:
  time.sleep( 20 )
  top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
  top_10_df = top_10_tweets.toPandas()
  display.clear_output(wait=True)
  sn.plt.figure( figsize = ( 10, 8 ) )
  sn.barplot( x="count", y="tag", data=top_10_df)
  sn.plt.show()
  count = count + 1

AnalysisException: 'Table or view not found: tweets; line 1 pos 23'